# Anime Dataset

This part of the notebook cleans the **Anime Recommendation** dataset to prepare it for NLP-based recommendations.
It follows the proposal steps: normalize text (stopword removal + lemmatization) for *synopsis/plot* and clean/standardize the *genre* field; coerce numeric fields (*episodes*, *score*); and remove duplicates/missing-critical rows.


In [ ]:
# %% [setup]
import pandas as pd
import numpy as np
import re
import os
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Text processing
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Ensure NLTK data is available
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

DATASET_PATH = "/content/anime_recommendation_dataset.csv"
OUTPUT_CSV   = "/mnt/data/cleaned_anime_dataset.csv"

In [ ]:
# %% [load]
df = pd.read_csv(DATASET_PATH)
print("Shape:", df.shape)
print("Columns:", list(df.columns))
df.head(3)

Shape: (200, 6)
Columns: ['title', 'genre_clean', 'genre_tokens', 'clean_synopsis', 'score', 'episodes']


title                           genre_clean  \
0                     Cowboy Bebop         action,adventure,drama,sci fi   
1  Cowboy Bebop: Tengoku no Tobira           action,drama,mystery,sci fi   
2                           TRIGUN  action,adventure,comedy,drama,sci fi   

                                        genre_tokens  \
0         ['action', 'adventure', 'drama', 'sci fi']   
1           ['action', 'drama', 'mystery', 'sci fi']   
2  ['action', 'adventure', 'comedy', 'drama', 'sc...   

                                      clean_synopsis  score  episodes  
0  enter world distant future bounty hunter roam ...     86        26  
1  cowboy bebop crew travel star learn largest bo...     82         1  
2  vash stampede wanted man habit turning entire ...     80        26

In [ ]:
# %% [resolve_columns]
def resolve_col(df, candidates):
    cols = {c.lower(): c for c in df.columns}
    # exact
    for cand in candidates:
        if cand.lower() in cols:
            return cols[cand.lower()]
    # contains
    for c in df.columns:
        lc = c.lower()
        for cand in candidates:
            if cand.lower() in lc:
                return c
    return None

col_title     = resolve_col(df, ["title", "name"])
col_genre     = resolve_col(df, ["genre", "genres"])
col_synopsis  = resolve_col(df, ["synopsis", "plot", "description", "summary"])
col_chars     = resolve_col(df, ["characters", "cast", "people"])
col_episodes  = resolve_col(df, ["episodes", "eps", "num_episodes"])
col_score     = resolve_col(df, ["score", "rating", "rank", "average_score"])

print({
    "title": col_title, "genre": col_genre, "synopsis": col_synopsis,
    "characters": col_chars, "episodes": col_episodes, "score": col_score
})

{'title': 'title', 'genre': 'genre_clean', 'synopsis': 'clean_synopsis', 'characters': None, 'episodes': 'episodes', 'score': 'score'}


In [ ]:
# %% [basic_cleaning]
required = [col_title, col_genre, col_synopsis]
required = [c for c in required if c is not None]
if required:
    before = len(df)
    df = df.dropna(subset=required)
    df = df[df[required].applymap(lambda x: str(x).strip() != "").all(axis=1)]
    print(f"Dropped {before - len(df)} rows missing required fields.")

if col_title:
    before = len(df)
    df = df.drop_duplicates(subset=[col_title])
    print(f"Dropped {before - len(df)} duplicate titles.")

Dropped 0 rows missing required fields.
Dropped 0 duplicate titles.


/tmp/ipython-input-3034923210.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df[df[required].applymap(lambda x: str(x).strip() != "").all(axis=1)]


In [ ]:
# %% [numerics]
if col_episodes:
    df[col_episodes] = pd.to_numeric(df[col_episodes], errors='coerce').fillna(0).astype(int)
if col_score:
    # keep as float; fill with column mean if missing
    df[col_score] = pd.to_numeric(df[col_score], errors='coerce')
    if df[col_score].notna().any():
        mean_score = df[col_score].mean()
        df[col_score] = df[col_score].fillna(mean_score)
    else:
        # if all NaN, set to 0
        df[col_score] = 0.0

In [ ]:
# %% [text_helpers]
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text: str) -> str:
    text = re.sub(r'[^a-zA-Z\s]', ' ', str(text))
    tokens = []
    for w in text.split():
        lw = w.lower()
        if lw not in stop_words:
            tokens.append(lemmatizer.lemmatize(lw))
    return " ".join(tokens)

In [ ]:
# %% [clean_synopsis]
if True and col_synopsis is not None:
    df['clean_synopsis'] = df[col_synopsis].apply(clean_text)
else:
    df['clean_synopsis'] = ""  # fallback
df['clean_synopsis'].head(3)

0    enter world distant future bounty hunter roam ...
1    cowboy bebop crew travel star learn largest bo...
2    vash stampede wanted man habit turning entire ...
Name: clean_synopsis, dtype: object

In [ ]:
# %% [clean_genres]
def clean_genre_field(s: str) -> str:
    s = str(s).lower()
    # keep letters, commas, and spaces; then normalize spaces
    s = re.sub(r'[^a-z,\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    # also normalize comma spacing
    s = re.sub(r'\s*,\s*', ',', s)
    return s

def split_genres(s: str):
    if not s:
        return []
    return [g.strip() for g in s.split(',') if g.strip()]

if col_genre is not None:
    df['genre_clean'] = df[col_genre].apply(clean_genre_field)
    df['genre_tokens'] = df['genre_clean'].apply(split_genres)
else:
    df['genre_clean'] = ""
    df['genre_tokens'] = [[]]

df[['genre_clean', 'genre_tokens']].head(5)

genre_clean  \
0         action,adventure,drama,sci fi   
1           action,drama,mystery,sci fi   
2  action,adventure,comedy,drama,sci fi   
3     action,drama,mystery,supernatural   
4        adventure,fantasy,supernatural   

                                 genre_tokens  
0          [action, adventure, drama, sci fi]  
1            [action, drama, mystery, sci fi]  
2  [action, adventure, comedy, drama, sci fi]  
3      [action, drama, mystery, supernatural]  
4          [adventure, fantasy, supernatural]

In [ ]:
# %% [finalize_columns]
out_cols = []
if col_title: out_cols.append(col_title)
if col_genre: out_cols.append('genre_clean'); out_cols.append('genre_tokens')
out_cols.append('clean_synopsis')
if col_score: out_cols.append(col_score)
if col_episodes: out_cols.append(col_episodes)

# Rename standard output names
rename_map = {}
if col_title: rename_map[col_title] = 'title'
if col_score: rename_map[col_score] = 'score'
if col_episodes: rename_map[col_episodes] = 'episodes'

df_out = df[out_cols].rename(columns=rename_map)

print("Output columns:", list(df_out.columns))
df_out.head(5)

Output columns: ['title', 'genre_clean', 'genre_tokens', 'clean_synopsis', 'score', 'episodes']


title                           genre_clean  \
0                     Cowboy Bebop         action,adventure,drama,sci fi   
1  Cowboy Bebop: Tengoku no Tobira           action,drama,mystery,sci fi   
2                           TRIGUN  action,adventure,comedy,drama,sci fi   
3               Witch Hunter ROBIN     action,drama,mystery,supernatural   
4                   Bouken Ou Beet        adventure,fantasy,supernatural   

                                 genre_tokens  \
0          [action, adventure, drama, sci fi]   
1            [action, drama, mystery, sci fi]   
2  [action, adventure, comedy, drama, sci fi]   
3      [action, drama, mystery, supernatural]   
4          [adventure, fantasy, supernatural]   

                                      clean_synopsis  score  episodes  
0  enter world distant future bounty hunter roam ...     86        26  
1  cowboy bebop crew travel star learn largest bo...     82         1  
2  vash stampede wanted man habit turning entire ...     80        26  
3  robin sena powerful craft user drafted stnj gr...     68        26  
4  dark century people suffering rule devil vande...     65        52

In [ ]:
# %% [save]
OUTPUT_CSV   = "/content/anime_recommendation_dataset.csv"
df_out.to_csv(OUTPUT_CSV, index=False)
print("Saved cleaned dataset to:", OUTPUT_CSV)

Saved cleaned dataset to: /content/anime_recommendation_dataset.csv


### Preview of Cleaned Data


In [ ]:
# %% [preview]
df_out.head(500)

title  \
0                                     Cowboy Bebop   
1                  Cowboy Bebop: Tengoku no Tobira   
2                                           TRIGUN   
3                               Witch Hunter ROBIN   
4                                   Bouken Ou Beet   
..                                             ...   
195                          Kidou Senkan Nadesico   
196  Kidou Senkan Nadesico: The prince of darkness   
197                                Kuro no Danshou   
198                                    Mezzo Forte   
199                                          MEZZO   

                                genre_clean  \
0             action,adventure,drama,sci fi   
1               action,drama,mystery,sci fi   
2      action,adventure,comedy,drama,sci fi   
3         action,drama,mystery,supernatural   
4            adventure,fantasy,supernatural   
..                                      ...   
195      action,comedy,mecha,romance,sci fi   
196        action,comedy,drama,mecha,sci fi   
197                   hentai,horror,mystery   
198             action,comedy,hentai,sci fi   
199  action,adventure,comedy,mystery,sci fi   

                                     genre_tokens  \
0              [action, adventure, drama, sci fi]   
1                [action, drama, mystery, sci fi]   
2      [action, adventure, comedy, drama, sci fi]   
3          [action, drama, mystery, supernatural]   
4              [adventure, fantasy, supernatural]   
..                                            ...   
195      [action, comedy, mecha, romance, sci fi]   
196        [action, comedy, drama, mecha, sci fi]   
197                     [hentai, horror, mystery]   
198              [action, comedy, hentai, sci fi]   
199  [action, adventure, comedy, mystery, sci fi]   

                                        clean_synopsis  score  episodes  
0    enter world distant future bounty hunter roam ...     86        26  
1    cowboy bebop crew travel star learn largest bo...     82         1  
2    vash stampede wanted man habit turning entire ...     80        26  
3    robin sena powerful craft user drafted stnj gr...     68        26  
4    dark century people suffering rule devil vande...     65        52  
..                                                 ...    ...       ...  
195  akito want fight despite childhood spent anime...     73        26  
196  two year passed since end martian successor na...     64         1  
197  based erotic game abogado power oz club br br ...     45         4  
198  mikura member mercenary team asked kidnap base...     64         2  
199  mikura kurokawa harada member danger service a...     62        13  

[200 rows x 6 columns]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

### TF-IDF Vectorization

In [ ]:
tv = TfidfVectorizer(min_df=0., max_df=1., norm='l2', use_idf=True, smooth_idf=True)
tv_matrix = tv.fit_transform(df_out['clean_synopsis'])
tv_matrix = tv_matrix.toarray()

print("Shape of TF-IDF matrix:", tv_matrix.shape)

Shape of TF-IDF matrix: (200, 4127)


In [ ]:
vocab = tv.get_feature_names_out()
tv_df = pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)
display(tv_df.head())

abandoned  abduction  abel  ability  able  aboard  abogado  abroad  \
0        0.0        0.0   0.0      0.0  0.00     0.0      0.0     0.0   
1        0.0        0.0   0.0      0.0  0.00     0.0      0.0     0.0   
2        0.0        0.0   0.0      0.0  0.00     0.0      0.0     0.0   
3        0.0        0.0   0.0      0.0  0.00     0.0      0.0     0.0   
4        0.0        0.0   0.0      0.0  0.06     0.0      0.0     0.0   

   absolute  absolutely  ...  zenon  zentraedi  zeon  zeonic  zephyranthes  \
0       0.0         0.0  ...   0.00        0.0   0.0     0.0           0.0   
1       0.0         0.0  ...   0.00        0.0   0.0     0.0           0.0   
2       0.0         0.0  ...   0.00        0.0   0.0     0.0           0.0   
3       0.0         0.0  ...   0.00        0.0   0.0     0.0           0.0   
4       0.0         0.0  ...   0.38        0.0   0.0     0.0           0.0   

   zero  zeta  zodiac  zombie   zz  
0   0.0   0.0     0.0     0.0  0.0  
1   0.0   0.0     0.0     0.0  0.0  
2   0.0   0.0     0.0     0.0  0.0  
3   0.0   0.0     0.0     0.0  0.0  
4   0.0   0.0     0.0     0.0  0.0  

[5 rows x 4127 columns]

### Cosine Similarity

In [ ]:
cosine_matrix = cosine_similarity(tv_matrix)
cosine_df = pd.DataFrame(cosine_matrix)
cosine_df

0         1         2         3         4         5         6    \
0    1.000000  0.065268  0.075272  0.055293  0.015275  0.007480  0.026220   
1    0.065268  1.000000  0.039531  0.042057  0.019254  0.023537  0.019337   
2    0.075272  0.039531  1.000000  0.080371  0.007710  0.031259  0.032252   
3    0.055293  0.042057  0.080371  1.000000  0.019756  0.072954  0.023073   
4    0.015275  0.019254  0.007710  0.019756  1.000000  0.000000  0.019759   
..        ...       ...       ...       ...       ...       ...       ...   
195  0.003618  0.069737  0.017870  0.012915  0.003226  0.016334  0.045704   
196  0.027621  0.052433  0.016668  0.018598  0.026320  0.028744  0.014227   
197  0.018430  0.009913  0.033866  0.035628  0.004694  0.024497  0.034868   
198  0.009987  0.022178  0.030736  0.012317  0.003619  0.059407  0.032049   
199  0.023114  0.053981  0.043605  0.030465  0.007454  0.002881  0.029206   

          7         8         9    ...       190       191       192  \
0    0.004967  0.000000  0.018240  ...  0.007416  0.014743  0.005264   
1    0.016665  0.018954  0.003542  ...  0.024879  0.009853  0.033622   
2    0.012574  0.000000  0.038836  ...  0.013236  0.028092  0.015982   
3    0.012714  0.000000  0.013734  ...  0.010363  0.031196  0.025991   
4    0.014961  0.017815  0.021668  ...  0.000000  0.000000  0.000000   
..        ...       ...       ...  ...       ...       ...       ...   
195  0.031789  0.000000  0.021687  ...  0.017918  0.020335  0.051507   
196  0.011376  0.010005  0.034302  ...  0.005884  0.004578  0.004177   
197  0.013380  0.000000  0.006243  ...  0.034844  0.025082  0.034248   
198  0.075665  0.086950  0.009839  ...  0.027548  0.021164  0.046359   
199  0.012894  0.000000  0.025128  ...  0.035099  0.016027  0.056112   

          193       194       195       196       197       198       199  
0    0.015096  0.020667  0.003618  0.027621  0.018430  0.009987  0.023114  
1    0.028971  0.018947  0.069737  0.052433  0.009913  0.022178  0.053981  
2    0.018835  0.014295  0.017870  0.016668  0.033866  0.030736  0.043605  
3    0.037244  0.007892  0.012915  0.018598  0.035628  0.012317  0.030465  
4    0.000000  0.040402  0.003226  0.026320  0.004694  0.003619  0.007454  
..        ...       ...       ...       ...       ...       ...       ...  
195  0.017104  0.043954  1.000000  0.162405  0.027741  0.019699  0.034973  
196  0.010640  0.020585  0.162405  1.000000  0.007735  0.031191  0.020944  
197  0.030046  0.055384  0.027741  0.007735  1.000000  0.060997  0.018014  
198  0.017235  0.033485  0.019699  0.031191  0.060997  1.000000  0.106300  
199  0.020834  0.028404  0.034973  0.020944  0.018014  0.106300  1.000000  

[200 rows x 200 columns]